# **Day_03**

## *Part 1/2*

### Instructions

--- Day 3: Crossed Wires ---

The gravity assist was successful, and you're well on your way to the Venus refuelling station. During the rush back on Earth, the fuel management system wasn't completely installed, so that's next on the priority list.

Opening the front panel reveals a jumble of wires. Specifically, two wires are connected to a central port and extend outward on a grid. You trace the path each wire takes as it leaves the central port, one wire per line of text (your puzzle input).

The wires twist and turn, but the two wires occasionally cross paths. To fix the circuit, you need to find the intersection point closest to the central port. Because the wires are on a grid, use the Manhattan distance for this measurement. While the wires do technically cross right at the central port where they both start, this point does not count, nor does a wire count as crossing with itself.

For example, if the first wire's path is R8,U5,L5,D3, then starting from the central port (o), it goes right 8, up 5, left 5, and finally down 3:

```
...........
...........
...........
....+----+.
....|....|.
....|....|.
....|....|.
.........|.
.o-------+.
...........
```

Then, if the second wire's path is U7,R6,D4,L4, it goes up 7, right 6, down 4, and left 4:

```
...........
.+-----+...
.|.....|...
.|..+--X-+.
.|..|..|.|.
.|.-X--+.|.
.|..|....|.
.|.......|.
.o-------+.
...........
```

These wires cross at two locations (marked X), but the lower-left one is closer to the central port: its distance is 3 + 3 = 6.

Here are a few more examples:

    R75,D30,R83,U83,L12,D49,R71,U7,L72
    U62,R66,U55,R34,D71,R55,D58,R83 = distance 159
    R98,U47,R26,D63,R33,U87,L62,D20,R33,U53,R51
    U98,R91,D20,R16,D67,R40,U7,R15,U6,R7 = distance 135

What is the Manhattan distance from the central port to the closest intersection?


### Imports

In [1]:
import pathlib

### Inputs

In [2]:
# Define file path for inputs
path = pathlib.Path("inputs/day_03.txt")

# Read and convert to list
diagram_raw = path.read_text().split("\n")

# Split out the wires to individual lists
wire_1 = diagram_raw[0].split(',')
wire_2 = diagram_raw[1].split(',')

### Solution

#### Define the tracer function

In [3]:
# Define our function
def trace_wires(wire):
    global position_history
    # Set position to center
    current_position = [0,0]
    # Save as to find crossed paths later
    position_history = []
    
    # Iterate through each wire segment
    for ix,segment in enumerate(wire):
        #print('='*20,f"Index {ix}",f"Segment {segment}",'='*20)

        direction = segment[0]
        distance = int(segment[1:])

        #print(f"Dir:{direction} Dist:{distance}")

        for i in range(distance):
            # If up
            if direction == 'U':
                step = tuple([current_position[0]+1,current_position[1]])

            # If down
            elif direction == 'D':
                step = tuple([current_position[0]-1,current_position[1]])

            # If left
            elif direction == 'L':
                step = tuple([current_position[0],current_position[1]-1])

            # If right
            elif direction == 'R':
                step = tuple([current_position[0],current_position[1]+1])
            
            #print("step:",step)
            #print("current pos:",current_position)
            current_position = step
            position_history.append(current_position)
            
        #position_history.append(tuple(current_position))
        #print(f"Current position is now {current_position}")
        
    return position_history
    


#### Trace the history/paths of each wire

In [5]:
wire_histories = []

for wire in [wire_1,wire_2]:
    wire_history = trace_wires(wire)
    wire_histories.append(wire_history)
    
print(f"Traced {len(wire_histories[0])+len(wire_histories[1]):,} wire segments.")

Traced 288,956 wire segments.


#### Compare paths for closest to origin

In [6]:
wire_1_history = set(wire_histories[1]) 

shortest_crossed_path = 1e9
for ix,position in enumerate(wire_histories[0]):
    if position == (0,0):
        continue
    if position in wire_1_history:
        distance = abs(position[1] - 0) + abs(position[0] - 0)
        if distance < shortest_crossed_path:
            shortest_crossed_path = distance
            print(f"Found new shortest path: {position}")
            print(f"Distance: {distance}")

Found new shortest path: (1383, 2487)
Distance: 3870
Found new shortest path: (1428, 1381)
Distance: 2809
Found new shortest path: (1159, 970)
Distance: 2129


## *Part 2/2*

### Instructions

--- Part Two ---

It turns out that this circuit is very timing-sensitive; you actually need to minimize the signal delay.

To do this, calculate the number of steps each wire takes to reach each intersection; choose the intersection where the sum of both wires' steps is lowest. If a wire visits a position on the grid multiple times, use the steps value from the first time it visits that position when calculating the total value of a specific intersection.

The number of steps a wire takes is the total number of grid squares the wire has entered to get to that location, including the intersection being considered. Again consider the example from above:

```
...........
.+-----+...
.|.....|...
.|..+--X-+.
.|..|..|.|.
.|.-X--+.|.
.|..|....|.
.|.......|.
.o-------+.
...........
```

In the above example, the intersection closest to the central port is reached after 8+5+5+2 = 20 steps by the first wire and 7+6+4+3 = 20 steps by the second wire for a total of 20+20 = 40 steps.

However, the top-right intersection is better: the first wire takes only 8+5+2 = 15 and the second wire takes only 7+6+2 = 15, a total of 15+15 = 30 steps.

Here are the best steps for the extra examples from above:

    R75,D30,R83,U83,L12,D49,R71,U7,L72
    U62,R66,U55,R34,D71,R55,D58,R83 = 610 steps
    R98,U47,R26,D63,R33,U87,L62,D20,R33,U53,R51
    U98,R91,D20,R16,D67,R40,U7,R15,U6,R7 = 410 steps

What is the fewest combined steps the wires must take to reach an intersection?


### Solution

We will just make some slight modifications to the function to keep track of total steps

In [7]:
# Define our function
def trace_wires(wire):
    global position_history
    # Set position to center
    current_position = [0,0]
    # Save as to find crossed paths later
    position_history = []
    # Keep track of total distance traveled
    total_distance = 0
    # Dict to link distances to positions
    total_distance_dict = {}
    
    # Iterate through each wire segment
    for ix,segment in enumerate(wire):
        #print('='*20,f"Index {ix}",f"Segment {segment}",'='*20)

        direction = segment[0]
        distance = int(segment[1:])
        #print(f"Dir:{direction} Dist:{distance}")

        for i in range(distance):
            # If up
            if direction == 'U':
                step = tuple([current_position[0]+1,current_position[1]])

            # If down
            elif direction == 'D':
                step = tuple([current_position[0]-1,current_position[1]])

            # If left
            elif direction == 'L':
                step = tuple([current_position[0],current_position[1]-1])

            # If right
            elif direction == 'R':
                step = tuple([current_position[0],current_position[1]+1])
            
            #print("step:",step)
            #print("current pos:",current_position)
            current_position = step
            position_history.append(current_position)
            
            total_distance += 1
            total_distance_dict[total_distance] = current_position
            
        #position_history.append(tuple(current_position))
        #print(f"Current position is now {current_position}")
        
    return position_history, total_distance_dict
    


Trace the history/paths of each wire while also tracking the total steps elapsed at each segment/step

In [8]:
wire_histories = []
wire_distance_dicts = []

for wire in [wire_1,wire_2]:
    wire_history, wire_distance_dict = trace_wires(wire)
    wire_histories.append(wire_history)
    wire_distance_dicts.append(wire_distance_dict)
    
print(f"Traced {len(wire_histories[0])+len(wire_histories[1]):,} wire segments.")

Traced 288,956 wire segments.


Add up combined steps to each wire crossing and print results

In [38]:
for i in range(1,len(wire_distance_dicts[0])+1):
    position_1 = wire_distance_dicts[0][i]
    if position_1 == (0,0):
        continue
    if position_1 in wire_distance_dicts[1].values():
        print("="*20,"Found crossed path","="*20)
        print("Position:",position_1)
        print("Steps for wire 1: ",i)
        position_2 = [(k,v) for k,v in wire_distance_dicts[1].items() if v == position_1]
        position_2_distance = position_2[0][0]
        combined_distance = i + position_2_distance
        print("Steps for wire 2: ",position_2_distance)
        print("Combined distance: ",combined_distance)

==================== Found crossed path ====================
Position: (1383, 2487)
Steps for wire 1:  9610
Steps for wire 2:  125052
Combined distance:  134662
==================== Found crossed path ====================
Position: (1428, 1381)
Steps for wire 1:  14365
Steps for wire 2:  127047
Combined distance:  141412
==================== Found crossed path ====================
Position: (1428, 1906)
Steps for wire 1:  14890
Steps for wire 2:  125678
Combined distance:  140568
==================== Found crossed path ====================
Position: (1383, 2326)
Steps for wire 1:  15355
Steps for wire 2:  125213
Combined distance:  140568
==================== Found crossed path ====================
Position: (1383, 2546)
Steps for wire 1:  21603
Steps for wire 2:  124993
Combined distance:  146596
==================== Found crossed path ====================
Position: (1383, 2072)
Steps for wire 1:  22099
Steps for wire 2:  125467
Combined distance:  147566
==================== Found cr